# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
! python taxi_fare_prediction_champion.py --environment=conda --production --branch champion --production argo-workflows create

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-ovzw7jjg7psagpyw-0-pcke
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-ovzw7jjg7psagpyw-0-pcke")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-ovzw7jjg7psagpyw-0-pcke
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow taxifareprediction.prod.champion.taxifareprediction fo

In [59]:
! python taxi_fare_prediction_champion.py --environment=conda --production --branch champion --production argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifareprediction-rnvtw).
See the run in the UI at https://ui-pw-623054330.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.champion.taxifareprediction-rnvtw


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [60]:
! python taxi_fare_prediction_challenger.py --environment=conda --production --branch challenger --production argo-workflows create

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.challenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-6iffsxtybx6fkjku-0-crtx
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-6iffsxtybx6fkjku-0-crtx")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-6iffsxtybx6fkjku-0-crtx
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow taxifareprediction.prod.challenger.taxifarepredict

In [61]:
! python taxi_fare_prediction_challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.challenger.taxifareprediction-6qmf4).
See the run in the UI at https://ui-pw-623054330.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-6qmf4


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
    * Yes.
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model?
    * I would evaluate the models gain vs the added complexity in context of the current state of the project. In the beginning of a project it is ok to add a larger amount of complexity even for relatively small gains. But a some point it important to think about marginal utility and production robustness, e.g. runtime, maintability and robustness.
- What gives you confidence one model is better than another?
    * Better scores and while looking at certain fractions of the data -- clear improvements. If model performance is almost equal: Better maintability/robustness. 
- What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?
    * For new projects I proactively work on the model and validate it. I check if the metrices improve, if the feature a have a understandable channel in the model with reasonable importance scores and signs. To be preprared for discussions with stakeholders a test the model for certain subsets of the data that represent edge-cases. For my changes I monitor runtimes to do not fall in a complexity trap.
    * If a model is in production for a longer time, my approach is to work on changes in the model logic, only after I discussed observed weaknesses of the model with my stakeholders. The only exemption is when I observe severe missbehavior of the model in the online metrics. I then asses if the changes did improve the overall model performance by etablished metrics and, especially, if the weaknesses improved. Afterwards I come back to my stakeholders and present everything via a nice presentation.

In [94]:
from metaflow import Flow, namespace

# these values are unique to your deployment!
flow_name = "TaxiFarePrediction"
champion_namespace = "production:mfprj-ovzw7jjg7psagpyw-0-pcke" 
challenger_namespace = "production:mfprj-6iffsxtybx6fkjku-0-crtx"

best_score = -1; winner = None; winner_namespace = None

df_scores = []

for n in [champion_namespace, challenger_namespace]:
    namespace(n)
    run = Flow(flow_name).latest_successful_run
    df_scrs = pd.DataFrame({"MODEL": run.data.model_name, 
                            "AVERAGE_CV_R2": run.data.score,
                            "NAMESPACE": n,
                            "WINNER": False},
                            index=[0])
    df_scores.append(df_scrs)

if len(df_scores) > 0:
    df_scores = pd.concat(df_scores)
    df_scores["WINNER"] = df_scores["AVERAGE_CV_R2"] == df_scores["AVERAGE_CV_R2"].max()
    df_scores = df_scores.sort_values("AVERAGE_CV_R2", ascending=False).reset_index(drop=True)

df_scores

,MODEL,AVERAGE_CV_R2,NAMESPACE,WINNER
0,Lasso,0.919775,production:mfprj-6iffsxtybx6fkjku-0-crtx,True
1,Linear Regression,0.897537,production:mfprj-ovzw7jjg7psagpyw-0-pcke,False


In [96]:
namespace(df_scores.loc[0, "NAMESPACE"])
print(df_scores.loc[0, "MODEL"])
bst = run.data.model
bst.coef_


Lasso


/home/workspace/mambaforge/envs/full-stack-metaflow-corise/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 1.3.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([ 2.67173295e-01,  4.90305685e-03,  4.20436011e-02, -3.04051882e-02,
        5.76264714e-02,  3.30274826e-01,  1.67292404e-01,  5.69398792e-02,
       -2.00374295e-03,  3.78158482e+00])

In [97]:
flow = Flow(flow_name)
list(flow.runs())

[Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-bmgdl'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-6qmf4'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-vtgqg'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-bt28z'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-nf9pm'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-rg8s6'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-smfrj'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-xtb7f'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-9fm82'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-j9phd'),
 Run('TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxif

In [98]:
n

'production:mfprj-6iffsxtybx6fkjku-0-crtx'

## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!